In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir gdrive
!google-drive-ocamlfuse gdrive
!ls gdrive

In [5]:
cd gdrive

/content/gdrive


In [0]:
file= open('poemShakespeare.txt').read()

In [0]:
raw_text= file.lower()

In [0]:
tok= raw_text.split()

In [0]:
text=""
for i in tok:
  text+= i+ " "

In [0]:
chars = sorted(list(set(text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [0]:
seq_length=100
vocab= len(chars)

In [27]:
len(text)

14544

In [0]:
sequences=[]
y=[]
for i in range(len(text)):
  if i+seq_length>=len(text):
    break
  sequences.append(text[i:i+seq_length])
  y.append(text[i+seq_length])
  

In [29]:
len(y)

14444

In [0]:
X=[]
for seq in sequences:
  tmp=[]
  for i in seq:
    tmp.append(char_to_int[i])
  X.append(tmp)
  
Y=[]
for i in y:
  Y.append(char_to_int[i])

In [0]:
X[:6]

In [0]:
from keras.utils import np_utils
datax = np_utils.to_categorical(X)
datay = np_utils.to_categorical(Y)

In [33]:
len(datax[0][0])

45

In [34]:
datay.shape

(14444, 45)

In [35]:
datax.shape

(14444, 100, 45)

In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint

In [0]:
model = Sequential()
model.add(LSTM(256, input_shape=(datax.shape[1], datax.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(datay.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

filepath="weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(datax, datay, epochs=50, batch_size=64, callbacks=callbacks_list)

seq length of 10 gave 0.8964 model loss error . Seq length of 100 gave 0.09195

In [47]:
tstr="“A Lover’s Complaint” begins with a young woman weeping at the edge of a river, into which she throws torn-up letters, rings, and other tokens of love. An old man nearby approaches her and asks the reason for her sorrow. She responds by telling him of a former lover who pursued, seduced, and finally abandoned her. She recounts in detail the speech her lover gave to her which seduced her. She concludes her story by conceding that she would fall for the young man's false charms again."
tstr=tstr.lower()
tmp=""
for i in tstr:
  if i in chars:
    tmp+=i
tstr=tmp
tstr= tstr[:seq_length]
tstr=tstr.lower()
tstr


'a lovers complaint begins with a young woman weeping at the edge of a river, into which she throws t'

In [48]:

print(tstr,end='')
for i in range(1000):
  test=[]
  for i in tstr:
    test.append(char_to_int[i])

  test= np_utils.to_categorical(test,num_classes=vocab)
  test= test.reshape(1,test.shape[0],test.shape[1])
  ans= model.predict(test)
  pp=chars[numpy.argmax(ans)]
  print(pp,end='')
  tstr+=pp
  tstr=tstr[1:]
  
  

a lovers complaint begins with a young woman weeping at the edge of a river, into which she throws that strong-bonded oath that shall prefer and undertake my troth.' 'this said, his watery eyes he did dismount, whose sights till then were levell'd on my face; each cheek a river running from a fount 285 with brinish current downward flow'd apace: o, how the channel to the stream gave grace! who glazed with crystal gate the glowing roses that flame through water which their hue encloses. 'o father, what a hell of witchcraft lies 290 in the small orb of one particular tear! but with the inundation of the eyes what rocky heart to water will not wear? what breast so cold that is not warmed here? o cleft effect! cold modesty, hot wrath, 295 both fire from hence and chill extincture hath. 'for, lo, his passion, but an art of craft, even there mentloved him, was he such a storm as oft 'twixt may and april is to see, when winds breathe sweet, untidy though they be. his rudeness so with his auth